## 휴먼지능정보공학과 201910787 김영서

# 문제 1:서울시 지하철 호선별 역별 유/무임 승하차 인원 분석
'서울시 지하철 호선별 역별 유/무임 승하차 인원 정보'를 csv파일로 내려받아서 분석하세요

(https://data.seoul.go.kr/dataList/OA-12251/S/1/datasetView.do?tab=S)

In [1]:
import sys
import os

home=os.path.expanduser("~")
os.environ["PYSPARK_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.9-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### 문제 1-1의 답
'서울시 지하철 호선별 역별 유_무임 승하차 인원 정보.csv'를 데이터프레임으로 읽고, 스키마를 출력<br>
(한글이 깨질 경우에는 엑셀에서 UTF로 저장하거나, 직접 csv파일을 수정하거나 우회적인 방법을 통해서 해결, 약간의 감점).

In [50]:
subway = spark.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true').load('data/서울시 지하철 호선별 역별 유_무임 승하차 인원 정보.csv')

In [51]:
subway.printSchema()

root
 |-- 사용월: integer (nullable = true)
 |-- 호선명: string (nullable = true)
 |-- 지하철역: string (nullable = true)
 |-- 유임승차인원: integer (nullable = true)
 |-- 무임승차인원: integer (nullable = true)
 |-- 유임하차인원: integer (nullable = true)
 |-- 무임하차인원: integer (nullable = true)
 |-- 작업일자: integer (nullable = true)



### 문제 1-2의 답
건수를 출력

In [52]:
subway.count()

55171

### 문제 1-3의 답
'호선명'별 '유임승차인원'의 순위를 매기고, 그 결과를 30줄 출력 (마지막 'rank' 컬럼)

```
+------+------+--------+------------+------------+------------+------------+--------+----+
|사용월|호선명|지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+------+--------+------------+------------+------------+------------+--------+----+
|201605|일산선|    화정|      515925|      139163|      535460|      142410|20160608|   1|
|201512|일산선|    화정|      508044|      136734|      529106|      139931|20160108|   2|
|201603|일산선|    화정|      502144|      140596|      526030|      144088|20160408|   3|
```

In [58]:
import numpy as np
import pandas as pd

subway_3 = subway.sort_values(by="유임승차인원", ascending=False).groupby("호선명").head(3)

ValueError: should sort by at least one column

### 문제 1-4의 답
*  '호선명'별 '유임승차인원'이 순위가 가장 높은 경우만 30건 출력 (마지막 'rank' 컬럼이 모두 1)

```

+------+--------------+--------------------+------------+------------+------------+------------+--------+----+

|사용월|        호선명|            지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|

+------+--------------+--------------------+------------+------------+------------+------------+--------+----+

|201605|        일산선|                화정|      515925|      139163|      535460|      142410|20160608|   1|

|201905|        장항선|                아산|      109243|       12901|       93737|       11808|20190603|   1|

|201512|        경부선|              영등포|     1439049|      280623|     1545631|      280969|20160108|   1|

```

### 문제 1-5의 답
*  '호선명'별 '유임승차인원'의 zscore를 계산하고 출력. 윈도우 함수를 사용하여야 한다.

```

+------+------------+------------------+------------------+--------------------+

|호선명|유임승차인원|        meanByLine|          sdByLine|              zscore|

+------+------------+------------------+------------------+--------------------+

|일산선|      237714|231630.21716649432|127730.48519577553| 0.04762984203951724|

|일산선|      363144|231630.21716649432|127730.48519577553|  1.0296193788971475|

|일산선|      294566|231630.21716649432|127730.48519577553|  0.4927232738295993|

```



# 문제 2:회귀분석
sklearn의 make_regression함수를 다음과 같이 사용하여 데이터를 생성하고, 문제를 푸세요.

#### 데이터 생성([식-1])

In [91]:
from sklearn.datasets import make_regression
X, y, coef = make_regression(n_samples = 200,
                            n_features = 4,
                            n_informative = 3,
                            n_targets = 1,
                            noise = 0.0,
                            coef = True) 

### 문제 2-1의 답
gradient 방법으로 회귀식을 계산하고, 계수를 출력.<br>
또한 [식-1]의 'coef'를 출력하고, gradient방법에서 도출된 계수와 일치하는지 비교하세요.<br>
주의: 반복이 많으면 감점 (예를 들어, 1000 이상이면 많다고 간주할 것임)

In [92]:
import numpy as np
alpha = 0.02
theta=np.array(np.ones([X.shape[1]]))

In [93]:
def gradientDescent(x,y,theta,alpha,numIterations):
    for i in range(numIterations):
        #i+=1
        h=np.dot(x,theta)
        error=h-y
        cost=np.sum((h-y)**2)/len(x) #full MSE
        gradient=np.dot(x.T,error)*2/len(x) #w0,w1
        theta-=alpha*gradient
        if i%100==0:
            print("Iteration {0} | theta{1} Cost {2:.5f}".format(i,theta,cost))

    return theta

**계수 출력**

In [94]:
w4,w3,w2,w1=gradientDescent(X, y, theta, alpha, 800) 

Iteration 0 | theta[2.50010749 3.36146554 1.1692096  3.72523866] Cost 9065.64546
Iteration 100 | theta[42.36685972 51.65031675  0.35911404 65.5072738 ] Cost 2.82336
Iteration 200 | theta[4.36741009e+01 5.19755354e+01 1.48224623e-02 6.64221573e+01] Cost 0.00271
Iteration 300 | theta[4.37235592e+01 5.19723322e+01 5.68656642e-04 6.64373979e+01] Cost 0.00000
Iteration 400 | theta[4.37254831e+01 5.19720505e+01 2.19257772e-05 6.64377042e+01] Cost 0.00000
Iteration 500 | theta[4.37255582e+01 5.19720380e+01 8.50744408e-07 6.64377120e+01] Cost 0.00000
Iteration 600 | theta[4.37255611e+01 5.19720375e+01 3.31046710e-08 6.64377122e+01] Cost 0.00000
Iteration 700 | theta[4.37255612e+01 5.19720375e+01 1.28960022e-09 6.64377122e+01] Cost 0.00000


**비교**

In [95]:
print("예측 결과:      %.8f  %.8f  %.8f  %.8f"%(w4,w3,w2,w1))
print("[식-1의] coef:  %.8f  %.8f  %.8f  %.8f"%(coef[0],coef[1],coef[2],coef[3]))

예측 결과:      43.72556121  51.97203751  0.00000000  66.43771223
[식-1의] coef:  43.72556121  51.97203751  0.00000000  66.43771223


### 문제 2-2의 답
스파크 데이터프레임을 생성 (이후 계속 스파크로 푸세요)

In [96]:
import pandas as pd
pdDf=pd.DataFrame(X)
pdDf['y']=y

In [97]:
Df=spark.createDataFrame(pdDf)

In [98]:
Df = Df.withColumnRenamed("0", "x0")
Df = Df.withColumnRenamed("1", "x1")
Df = Df.withColumnRenamed("2", "x2")
Df = Df.withColumnRenamed("3", "x3")

In [99]:
Df.show(5)

+--------------------+-------------------+--------------------+--------------------+-------------------+
|                  x0|                 x1|                  x2|                  x3|                  y|
+--------------------+-------------------+--------------------+--------------------+-------------------+
| -0.5397017192382667| 0.8329027547714076| -0.5100548423652854| -1.0662451781197946| -51.14999766625296|
|-0.00339115150069...|-1.1316601186140154| -0.5892035094858212| -0.9676262804386547|-123.24983850094662|
| -1.4816785384136577|0.18075465755420211| -1.4567734774332153|-0.21502567111289722| -69.67885144696598|
|  3.5150546124925293|0.38313853181543467|-0.08827401307422125| -1.6337495471987837|  65.06764348909356|
| 0.29853695010243364| 2.0183587461393824|  0.0979203116012738|  1.0599796258082859|  188.3745334938853|
+--------------------+-------------------+--------------------+--------------------+-------------------+
only showing top 5 rows



### 문제 2-3의 답
* 훈련, 테스트 데이터를 6:4 비율로 분리하고,

In [101]:
(trainDf, testDf) = Df.randomSplit([0.6, 0.4])

### 문제 2-4의 답
* 회귀모델링하고 계수와 절편 출력. 
다음과 같이 실제와 예측을 10줄 출력.


```
+-------------------+-------------------+
|                  y|         prediction|
+-------------------+-------------------+
|-109.97900025087853|-109.80947358240276|
|-145.43579783438383|-144.69608022770103|
|-133.86831015463093|-133.02142763705754|
```

In [102]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(inputCols = ["x0","x1","x2","x3"],
                                   outputCol = 'features')

In [103]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='y', maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [104]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[va,lr])

In [105]:
model = pipeline.fit(trainDf)
modelTrainDf = model.transform(trainDf)

**계수와 절편 출력**

In [106]:
print("Coefficients: {}".format(model.stages[-1].coefficients))
print("Intercept: {:.3f}".format(model.stages[-1].intercept))

Coefficients: [43.38175548926913,51.639773309331225,0.0,66.10944308055264]
Intercept: -0.024


**실제와 예측 출력**

In [134]:
Df.select('y','prediction').show(10)

,y,pridiction
0,-51.149998,-51.149998
1,-123.249839,-123.249839
2,-69.678851,-69.678851
3,65.067644,65.067644
4,188.374533,188.374533
5,-109.866574,-109.866574
6,10.629199,10.629199
7,44.292317,44.292317
8,-79.198037,-79.198037
9,-62.956010,-62.956010


### 문제 2-5의 답
* 테스트 데이터에 대해 R2를 구해서 출력하세요.

In [63]:
modelTestDf=model.transform(testDf)

In [64]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator=RegressionEvaluator(labelCol="y", predictionCol="prediction", metricName="rmse")

In [65]:
rmse=evaluator.evaluate(modelTestDf)

In [66]:
f"Root Mean Squared Error (RMSE) on test data: {rmse:.8f}"

'Root Mean Squared Error (RMSE) on test data: 0.52657348'

In [67]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator=RegressionEvaluator(labelCol="y", predictionCol="prediction", metricName="r2")

In [68]:
f"r2: {evaluator.evaluate(modelTestDf):.8f}"

'r2: 0.99998153'

# 문제 3:텍스트 분류

### 문제 3-1의 답
데이터프레임을 생성하세요.<br>
영화리뷰를 파일로 작성하자. ipynb에서 직접 하거나 편리한 에디터를 사용함.

- 리뷰는 한 줄씩 저장하며, 각각의 긍정, 부정을 구분하여 레이블을 지정하고 함께 저장함.

- 파일로 작성하지 못하면, 이후 문제를 포기하지 말고 리스트로 만들거나 자신이 할 수 있는 방법을 통해서 만들어도 됨.

In [74]:
revDf = spark\
    .read\
    .options(header='false', inferschema='true', delimiter='\t')\
    .csv(os.path.join('data', 'movie_review.txt'))  # 0=부정, 1=긍정

In [4]:
revDf = revDf.withColumnRenamed("_c0", "review")
revDf = revDf.withColumnRenamed("_c1", "label")

In [5]:
revDf.show(5)

+----------------------------------+-----+
|                            review|label|
+----------------------------------+-----+
|고구마 먹은 영화 유치해서 못보겠다|    0|
|노잼이네요 왜 배우들이 출연했는...|    0|
| 제발 뻘짓 좀 그만하셨으면 이건...|    0|
|         스토리가 매우 매우 아쉽다|    0|
|             영화 약간 이해가 안됨|    0|
+----------------------------------+-----+
only showing top 5 rows



In [6]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="review", outputCol="words")

In [7]:
tokDf = tokenizer.transform(revDf)

In [8]:
tokDf.show(5)

+----------------------------------+-----+------------------------------+
|                            review|label|                         words|
+----------------------------------+-----+------------------------------+
|고구마 먹은 영화 유치해서 못보겠다|    0|[고구마, 먹은, 영화, 유치해...|
|노잼이네요 왜 배우들이 출연했는...|    0|[노잼이네요, 왜, 배우들이, ...|
| 제발 뻘짓 좀 그만하셨으면 이건...|    0|[제발, 뻘짓, 좀, 그만하셨으...|
|         스토리가 매우 매우 아쉽다|    0|[스토리가, 매우, 매우, 아쉽다]|
|             영화 약간 이해가 안됨|    0|    [영화, 약간, 이해가, 안됨]|
+----------------------------------+-----+------------------------------+
only showing top 5 rows



### 문제 3-2의 답
불용어 제거, 불필요 단어 정리

- 느낌표, 점, '...', 'ㅋㅋㅋ', 'ㅠ', 'ㄹㅇ' 불필요한 단어 제거
- 불용어 제거: '그', '수', '본', '등', '때', '걸', '다', '게', '날','짱', '거', '중', '더', '넘', '두', '달', '올', '뻔', '단', '개', '듯'

In [22]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")

_mystopwords=['!', '.', ',', u'ㅋㅋㅋ', u'ㅠ', u'ㄹㅇ', u'그', u'수', u'본', u'등', u'때', u'걸', u'다', u'게', u'날', u'짱', u'거', u'중', u'더', u'넘', u'두', u'달', u'올', u'뻔', u'단', u'개', u'듯']
stop.setStopWords(_mystopwords)

StopWordsRemover_ae5b40ad3278

In [28]:
for e in stop.getStopWords():
    print (e, end="/")

!/./,/ㅋㅋㅋ/ㅠ/ㄹㅇ/그/수/본/등/때/걸/다/게/날/짱/거/중/더/넘/두/달/올/뻔/단/개/듯/

In [29]:
stopDf=stop.transform(tokDf) #불용어 제거 된 컬럼
stopDf.show()

+----------------------------------+-----+--------------------------------+--------------------------------+
|                            review|label|                           words|                         nostops|
+----------------------------------+-----+--------------------------------+--------------------------------+
|고구마 먹은 영화 유치해서 못보겠다|    0|  [고구마, 먹은, 영화, 유치해...|  [고구마, 먹은, 영화, 유치해...|
|노잼이네요 왜 배우들이 출연했는...|    0|  [노잼이네요, 왜, 배우들이, ...|  [노잼이네요, 왜, 배우들이, ...|
| 제발 뻘짓 좀 그만하셨으면 이건...|    0|  [제발, 뻘짓, 좀, 그만하셨으...|  [제발, 뻘짓, 좀, 그만하셨으...|
|         스토리가 매우 매우 아쉽다|    0|  [스토리가, 매우, 매우, 아쉽다]|  [스토리가, 매우, 매우, 아쉽다]|
|             영화 약간 이해가 안됨|    0|      [영화, 약간, 이해가, 안됨]|      [영화, 약간, 이해가, 안됨]|
|좋아하는 배우들인데 최근 영화라...|    0| [좋아하는, 배우들인데, 최근,...| [좋아하는, 배우들인데, 최근,...|
| 그지같네 도대체 이 따위 영화를...|    0|  [그지같네, 도대체, 이, 따위...|  [그지같네, 도대체, 이, 따위...|
|  스크린보다 시계를 더 많이 봄....|    0|  [스크린보다, 시계를, 더, 많...|  [스크린보다, 시계를, 많이, ...|
| 이때까지 본 영화중에 제일 최악...|    0|  [이때까지, 본, 영화중에, 제...|  [이때까지, 영화중

In [27]:
for r in stopDf.select("nostops").take(5):
    for e in r:
        print (e)

['고구마', '먹은', '영화', '유치해서', '못보겠다']
['노잼이네요', '왜', '배우들이', '출연했는지', '이해가', '잘', '안됨']
['제발', '뻘짓', '좀', '그만하셨으면', '이건', '여러모로', '낭비다.']
['스토리가', '매우', '매우', '아쉽다']
['영화', '약간', '이해가', '안됨']


### 문제 3-3의 답
TF-IDF를 계산

In [30]:
from pyspark.ml.feature import HashingTF, IDF
hashTF = HashingTF(inputCol="nostops", outputCol="features")
hashDf = hashTF.transform(stopDf)

In [31]:
idf = IDF(inputCol="features", outputCol="idf")

idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

In [32]:
for e in idfDf.select("nostops","idf").take(5):
    print(e)

Row(nostops=['고구마', '먹은', '영화', '유치해서', '못보겠다'], idf=SparseVector(262144, {114067: 3.1781, 145746: 1.9253, 202865: 2.7726, 232307: 2.7726, 243237: 2.7726}))
Row(nostops=['노잼이네요', '왜', '배우들이', '출연했는지', '이해가', '잘', '안됨'], idf=SparseVector(262144, {14123: 3.1781, 77545: 3.1781, 89636: 2.7726, 102360: 2.7726, 123815: 2.4849, 221856: 2.7726, 252373: 2.2618}))
Row(nostops=['제발', '뻘짓', '좀', '그만하셨으면', '이건', '여러모로', '낭비다.'], idf=SparseVector(262144, {45298: 2.7726, 104569: 3.1781, 159598: 3.1781, 160982: 3.1781, 165201: 3.1781, 196501: 3.1781, 249777: 2.7726}))
Row(nostops=['스토리가', '매우', '매우', '아쉽다'], idf=SparseVector(262144, {8121: 2.2618, 118336: 6.3561, 181130: 3.1781}))
Row(nostops=['영화', '약간', '이해가', '안됨'], idf=SparseVector(262144, {35471: 3.1781, 102360: 2.7726, 145746: 1.9253, 221856: 2.7726}))


In [33]:
idfDf.select("nostops", "features","idf").show(3)

+------------------------------+--------------------+--------------------+
|                       nostops|            features|                 idf|
+------------------------------+--------------------+--------------------+
|[고구마, 먹은, 영화, 유치해...|(262144,[114067,1...|(262144,[114067,1...|
|[노잼이네요, 왜, 배우들이, ...|(262144,[14123,77...|(262144,[14123,77...|
|[제발, 뻘짓, 좀, 그만하셨으...|(262144,[45298,10...|(262144,[45298,10...|
+------------------------------+--------------------+--------------------+
only showing top 3 rows



### 문제 3-4의 답
* 베이지안 모델을 적용해서 분류 (alpha는 1.0으로 설정)

In [72]:
from pyspark.ml.classification import NaiveBayes
nb=NaiveBayes(featuresCol='features', labelCol='label', modelType='multinomial', predictionCol='prediction',smoothing=1.0)

In [75]:
model=nb.fit(hashDf)

In [76]:
predictions=model.transform(hashDf)

In [79]:
predictions.select('label','review','prediction').show(45)

### 문제 3-5의 답
* 훈련, 테스트 데이터를 반반씩 분리하고, 훈련 후 정확성을 계산하고 출력.

In [125]:
(trainDf, testDf) = hashDf.randomSplit([0.5, 0.5])

In [83]:
model2=nb.fit(trainDf)

In [84]:
predictions_train=model2.transform(trainDf)

In [86]:
predictions_train.select('label','review','prediction').show(5)

In [87]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label")

In [89]:
evaluator.evaluate(predictions_train)

### 문제 3-6의 답
* 앞서 훈련한 베이지안 모델로 다음 문장을 긍정, 부정으로 분류하세요